In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
import math
from scipy.linalg import expm
import matplotlib.pyplot as plt
import torch
from cvxpylayers.torch import CvxpyLayer

In [2]:
# Parameter Initialization
# Background Parameter
g=[-3.7114, 0, 0]
pi = math.pi
m_dry=1505
m_wet=1905
I_sp=225
T1=930
T2=2480
n=6
phii=27*2*pi/360
alpha_val=6.293e-4
alpha = cp.Parameter(name="alpha", value=6.293e-4, nonneg=True)
#alphas = np.logspace(0, 0.01, 200)

In [3]:
# Optimal t_f and N
t_f=68
N=55
time_step=t_f/N
#Initial State
position_0=[3000, 1000, 2000]
velocity_0=[-50,10,100]
y0=np.array([position_0 + velocity_0 + [np.log(m_wet)]]).T
#print(y0, y0.shape)

In [4]:
# Define A_c Matrix
A_c1 = np.concatenate((np.zeros((3,3)), np.identity(3)), axis=1)
A_c2 = np.concatenate((A_c1, np.zeros((3,1))), axis=1)
A_c = np.concatenate((A_c2, np.zeros((4,7))), axis=0)
#print(A_c,A_c.shape)

In [5]:
# Define B_c Matrix
B_c1 = np.concatenate((np.zeros((3,3)), np.identity(3)),axis=0)
B_c2 = np.concatenate((B_c1, np.zeros((6,1))),axis=1)
B_c = np.concatenate((B_c2, np.array([[0,0,0,-alpha]])), axis=0)
print(B_c,B_c.shape)

[[0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0]
 [0.0 0.0 1.0 0.0]
 [0 0 0 Expression(CONSTANT, NONPOSITIVE, ())]] (7, 4)


In [6]:
# Define A & B Matrix
"""
  Work on this later!!!
  use SciPy
  
[A,B]=c2d(A_c,B_c,time_step);% A 7*7 ;B 7*4
"""



A = np.array([[1, 0, 0, 1.2364, 0, 0, 0],
              [0, 1, 0, 0, 1.2364, 0, 0],
              [0, 0, 1, 0, 0, 1.2364, 0],
              [0, 0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 1]

])
print("A=", A, A.shape)

B = np.array([[0.7643, 0, 0, 0],
          [0, 0.7643, 0, 0],
          [0, 0, 0.7643, 0],
          [1.2364, 0, 0, 0],
          [0, 1.2364, 0, 0],
          [0, 0, 1.2364, 0],
          [0, 0, 0, -alpha]])
print("B=", B, B.shape)
print(type(B))

A= [[1.     0.     0.     1.2364 0.     0.     0.    ]
 [0.     1.     0.     0.     1.2364 0.     0.    ]
 [0.     0.     1.     0.     0.     1.2364 0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.     0.     1.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     0.     0.     0.     1.    ]] (7, 7)
B= [[0.7643 0 0 0]
 [0 0.7643 0 0]
 [0 0 0.7643 0]
 [1.2364 0 0 0]
 [0 1.2364 0 0]
 [0 0 1.2364 0]
 [0 0 0 Expression(CONSTANT, NONPOSITIVE, ())]] (7, 4)
<class 'numpy.ndarray'>


In [7]:
# Matrix for State Computation  # A^k #(7*7N)
phi=np.zeros((7,7*N))
phi[:,0:7]=A
for k in range(2,N+1):
    phi[:,7*k-7:7*k]=A@phi[:,7*k-14:7*k-7]
print(phi, phi.shape)

[[ 1.     0.     0.    ...  0.     0.     0.   ]
 [ 0.     1.     0.    ... 68.002  0.     0.   ]
 [ 0.     0.     1.    ...  0.    68.002  0.   ]
 ...
 [ 0.     0.     0.    ...  1.     0.     0.   ]
 [ 0.     0.     0.    ...  0.     1.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     1.   ]] (7, 385)


In [8]:
# lambda matrix  #B+AB+......A^(k-1)B  #(7*4N)
lam=B  
lam_prev=B
for k in range(2,N+1):
    lam_next=A@lam_prev+B
    lam = np.concatenate([lam, lam_next], axis=1)
    lam_prev = lam_next

In [9]:
### testing lambda matrix
print(lam[:,8])
print(type(lam))

[6.87895488 0.0 0.0 3.7092 0.0 0.0 0.0]
<class 'numpy.ndarray'>


In [10]:
# psi matrix   #(7N*(4N+4))
psi = B 
psi = np.concatenate((psi, np.zeros((7,4*N))), axis=1)
psi_prev = psi
for k in range(2,N+1):
    psi_next=A@psi_prev
    psi = np.concatenate((psi, psi_next),axis=0)
    psi_prev = psi_next

In [11]:
### testinf psi matrix
print(type(psi))
print(psi[:14,:4])

<class 'numpy.ndarray'>
[[0.7643 0 0 0]
 [0 0.7643 0 0]
 [0 0 0.7643 0]
 [1.2364 0 0 0]
 [0 1.2364 0 0]
 [0 0 1.2364 0]
 [0 0 0 Expression(CONSTANT, NONPOSITIVE, ())]
 [2.29298496 0.0 0.0 Expression(CONSTANT, ZERO, ())]
 [0.0 2.29298496 0.0 Expression(CONSTANT, ZERO, ())]
 [0.0 0.0 2.29298496 Expression(CONSTANT, ZERO, ())]
 [1.2364 0.0 0.0 Expression(CONSTANT, ZERO, ())]
 [0.0 1.2364 0.0 Expression(CONSTANT, ZERO, ())]
 [0.0 0.0 1.2364 Expression(CONSTANT, ZERO, ())]
 [0.0 0.0 0.0 Expression(CONSTANT, NONPOSITIVE, ())]]


In [12]:
# upsilon matrix      #(4N*(4N+4))
upsilon=np.zeros((4*N,4*N+4)) 
for k in range(1,N+1):
    upsilon[4*k-4:4*k,4*k-4:4*k]=np.identity(4)
print(upsilon, upsilon.shape)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (220, 224)


In [13]:
# Matrix for Optimal Computation 
Z0 = np.array(cp.log(m_wet), ndmin=2)         #((N+1)*1)
for k in range(1,N+1):
    Z0_new = np.array(cp.log(m_wet-alpha*n*T2*math.cos(phii)*time_step*k), ndmin=2)
    Z0 = np.concatenate((Z0, Z0_new), axis=0)

In [14]:
print(Z0.shape)
print(Z0[7,0])
#print(Z0)

(56, 1)
log(1905.0 + -alpha @ 6.0 @ 2480.0 @ 0.8910065241883679 @ 1.2363636363636363 @ 7.0)


In [15]:
# Matrix for Optimal Computation 
mu_1 = np.array(n*T1*math.cos(phii)*cp.exp(-Z0[0,0]), ndmin = 2)  #(1*(N+1))
mu_2 = np.array(n*T2*math.cos(phii)*cp.exp(-Z0[0,0]), ndmin = 2)  #(1*(N+1))

for k in range(1, N+1):
    mu_1_new = np.array(n*T1*math.cos(phii)*cp.exp(-Z0[k,0]), ndmin = 2)
    mu_2_new = np.array(n*T2*math.cos(phii)*cp.exp(-Z0[k,0]), ndmin = 2)
    mu_1 = np.concatenate((mu_1, mu_1_new), axis = 1)
    mu_2 = np.concatenate((mu_2, mu_2_new), axis = 1)

In [16]:
print(f'mu_1 shape = {mu_1.shape}')
print(f'mu_2 shape = {mu_2.shape}')

print(mu_1[0,:5], end="\n---------------------------------------\n")
print(mu_2[0,:5])

mu_1 shape = (1, 56)
mu_2 shape = (1, 56)
[Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())]
---------------------------------------
[Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())]


In [17]:
# epsilon_k matrix      #(7*N)
epsilon_k = phi[:,7*0:7*(0+1)]@y0+lam[:,4*0:4*(0+1)]@np.array([g + [0]]).T  #(7,1)
for k in range(1,N):
    temp = phi[:,7*k:7*(k+1)]@y0+lam[:,4*k:4*(k+1)]@np.array([g + [0]]).T
    epsilon_k = np.concatenate((epsilon_k, temp), axis = 1)

In [18]:
epsilon_k.shape

(7, 55)

In [19]:
print(f'epsilon k shape = {epsilon_k.shape}')
print(epsilon_k[:,46])

epsilon k shape = (7, 55)
[Expression(CONSTANT, UNKNOWN, ()) Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONPOSITIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())
 Expression(CONSTANT, NONNEGATIVE, ())]


In [20]:
e_sigma = np.array([0,0,0,1])
#E = np.concatenate((np.identity(6), np.zeros((6,1))), axis=1)
F = np.concatenate((np.zeros(6), np.ones(1)), axis=None).reshape((1,7))
E_u = np.concatenate((np.identity(3), np.zeros((3,1))), axis=1)
#print(F,F.shape)

In [21]:
omega=np.zeros((1,4*N))  #1*4N
for k in range(0,N):
    omega[0,4*k:4*(k+1)]=e_sigma
omega=time_step*omega
#print(omega, omega.shape)

In [22]:
# Define and solve the CVXPY problem.
eta = cp.Variable((4*N+4,1))
obj = cp.Minimize(omega@eta[0:4*N,0])

In [23]:
def multiply(mat1, mat2):
    a = [[0]*mat2.shape[1]] * mat1.shape[0]
    for i in range(len(mat1)):
        for j in range(mat2.shape[1]):
            for k in range(mat2.shape[1]):
                a[i][j] += mat1[i][k] * mat2[k][j]
    a = np.array(a)
    return a 

In [24]:
#  Final height constraint
#h_const = [m@(epsilon_k[:,N-1]+psi[7*N-7:7*N,:]@eta) == np.zeros((7,1))]
m1 = np.concatenate((np.identity(3), np.zeros((3,4))), axis=1)
m2 = np.concatenate((np.zeros((4,3)), np.zeros((4,4))), axis=1)
m = np.concatenate((m1, m2), axis=0)
#print(f'm.shape = {m.shape}')
a = multiply(psi[7*N-7:7*N,:],eta)
#print(f'a.shape = {a.shape}')
b = epsilon_k[:,N-1, np.newaxis]+a
#print(f'b.shape = {b.shape}')
c = multiply(m,b)
#print(f'c.shape = {c.shape}')
h_const = [i == 0 for i in c[:,0]]
#print(h_const)

In [25]:
#  Final velocity constraint
#np.array([[0, 0, 0, 1, 0, 0, 0]])@(cp.reshape(epsilon_k[:,N-1], (7,1))+psi[7*N-7:7*N,:]@eta)==0,
#np.array([[0, 0, 0, 0, 1, 0, 0]])@(cp.reshape(epsilon_k[:,N-1], (7,1))+psi[7*N-7:7*N,:]@eta)==0,
#np.array([[0, 0, 0, 0, 0, 1, 0]])@(cp.reshape(epsilon_k[:,N-1], (7,1))+psi[7*N-7:7*N,:]@eta)==0,
a = multiply(psi[7*N-7:7*N,:],eta)
b = epsilon_k[:,N-1, np.newaxis]+a
d1 = np.array([[0, 0, 0, 1, 0, 0, 0]])
d2 = np.array([[0, 0, 0, 0, 1, 0, 0]])
d3 = np.array([[0, 0, 0, 0, 0, 1, 0]])
c1 = multiply(d1,b)
c2 = multiply(d2,b)
c3 = multiply(d3,b)
v_const = [
    c1[0,0]==0,
    c2[0,0]==0,
    c3[0,0]==0,
]

In [26]:
# Convexified Thrust Constraint
#Conv_const = []
#for k in range(N):
#    Conv_const.append(cp.norm(E_u@upsilon[4*k:4*k+4,:]@eta,2) <= e_sigma@upsilon[4*k:4*k+4,:]@eta) 
#    #cp.norm(E_u@upsilon[4*k:4*k+4,:]@eta,2)<=e_sigma@upsilon[4*k:4*k+4,:]@eta
Conv_const = [
      cp.SOC(e_sigma@upsilon[4*k:4*k+4,:]@eta, E_u@upsilon[4*k:4*k+4,:]@eta) for k in range(N)
]

In [27]:
def list_dot_product(v1, v2):
    v3 = 0
    for i in range(len(v1)):
        v3 += v1[i] * v2[i]
    return v3

In [28]:
# thrust constraints
thr_const = [mu_1[0,0]*(1-(F@y0-Z0[0,0])+0.5*(F@y0-Z0[0,0])**2)<=e_sigma@upsilon[0:4,:]@eta,
            e_sigma@upsilon[0:4,:]@eta<=mu_2[0,0]*(1-(F@y0-Z0[0,0]))]
N = 55
for k in range(9,10):
    h3 = multiply(psi[7*k:7*(k+1),:],eta)
    print(f'h3.shape = {h3.shape}')
#     h1 = multiply(F,epsilon_k[:,k, np.newaxis]+h3)[0,0]-Z0[k+1,0]
    h1 = list_dot_product(F.squeeze(),(epsilon_k[:,k, np.newaxis]+h3)[:,0])-Z0[k+1,0]
    print(f'h1.shape = {h1.shape}')
    h = 1-h1+0.5*(h1**2)
    print(f'h.shape = {h.shape}')
    thr_const.append(mu_1[0,k+1]*h<=e_sigma@upsilon[4*k:4*k+4,:]@eta)
    print(f'thr_const for iter:{k}')
    #---------
    #h2 = epsilon_k[:,k,np.newaxis]+h3
    #print(f'h2.shape = {h2.shape}')
    #print(f'F.shape = {F.shape}')
    #h4 = multiply(F,h2)
    #print(f'h4.shape = {h4.shape}')
    #h4[0,0]-Z0[k+1,0]
    
    #thr_const.append(e_sigma@upsilon[4*k:4*k+4,:]@eta<=mu_2[0,k+1]*(1-h2))

    
    
    
    
#thr_const.append(mu_1[0,k+1]*(1-(F.reshape((1,7))@(cp.reshape(epsilon_k[:,k],(7,1))+psi[7*k:7*(k+1),:]@eta)-Z0[k+1,0])+0.5*(F.reshape((1,7))@(cp.reshape(epsilon_k[:,k],(7,1))+psi[7*k:7*(k+1),:]@eta)-Z0[k+1,0])**2)<=e_sigma@upsilon[4*k:4*k+4,:]@eta)
#thr_const.append(e_sigma@upsilon[4*k:4*k+4,:]@eta<=mu_2[0,k+1]*(1-(F.reshape((1,7))@(cp.reshape(epsilon_k[:,k],(7,1))+psi[7*k:7*(k+1),:]@eta)-Z0[k+1,0])))

h3.shape = (7, 1)
h1.shape = ()
h.shape = ()
thr_const for iter:9


In [29]:
epsilon_k[:,k, np.newaxis]

array([[Expression(CONSTANT, UNKNOWN, ())],
       [Expression(CONSTANT, NONNEGATIVE, ())],
       [Expression(CONSTANT, NONNEGATIVE, ())],
       [Expression(CONSTANT, NONPOSITIVE, ())],
       [Expression(CONSTANT, NONNEGATIVE, ())],
       [Expression(CONSTANT, NONNEGATIVE, ())],
       [Expression(CONSTANT, NONNEGATIVE, ())]], dtype=object)

In [30]:
mu_1[0,k+1]*h<=e_sigma@upsilon[4*k:4*k+4,:]@eta

Inequality(Expression(CONVEX, UNKNOWN, ()))

In [31]:
mu_1[0,k+1]*h

Expression(CONVEX, UNKNOWN, ())

In [32]:
e_sigma@upsilon[4*k:4*k+4,:]@eta

Expression(AFFINE, UNKNOWN, (1,))

In [33]:
# Fuel mass constraints
f_const = []
for k in range(0,N):
    f_const.append(F.reshape((1,7))@(cp.reshape(epsilon_k[:,k],(7,1))+psi[7*k:7*(k+1),:]@eta)<=cp.log(m_wet-alpha*n*T1*math.cos(phii)*time_step*(k+1)))
    f_const.append(cp.log(m_wet-alpha*n*T2*math.cos(phii)*time_step*(k+1))<=F.reshape((1,7))@(cp.reshape(epsilon_k[:,k],(7,1))+psi[7*k:7*(k+1),:]@eta))


ValueError: setting an array element with a sequence.

In [ ]:
print(f_const[0])

In [ ]:
# Cone constraints
cone_const = [cp.norm(np.array([[0, 1, 0],[0, 0, 1]])@np.concatenate((np.identity(3),np.zeros((3,4))),axis=1)@(cp.reshape(epsilon_k[:,k],(7,1))+psi[7*k:7*(k+1),:]@eta),2)+np.array([[-0.268, 0, 0, 0, 0, 0, 0]])@(cp.reshape(epsilon_k[:,k],(7,1))+psi[7*k:7*(k+1),:]@eta)<=0]


In [ ]:
socp_constraints = constraints + Conv_const + thr_const + h_const + v_const + cone_const + f_const + cone_const

prob = cp.Problem(obj, socp_constraints)
prob.solve()
#print(obj.value)
print(prob.status)
print("Problem is dpp:",prob.is_dpp())
print("Problem is dcp:",prob.is_dcp())
#prob.solve()
#print(prob.parameters)

In [ ]:
cvxpylayer = CvxpyLayer(prob, parameters=[alpha], variables=[eta])
alpha_tch = torch.randn(requires_grad=True)
solution = cvxpylayer(alpha_tch)
solution.backward()

#print(obj.value)
#print(prob.status)
#print(alpha.value)
print(alpha.gradient)

#prob.solve(requires_grad=True)
#prob.backward()              #backpropagate through solution
#print(alpha.gradient)


In [ ]:
#     Output Information
# Converting output into manageable format
display_position = np.zeros((3,N+1))
display_velocity = np.zeros((3,N+1))
display_netthrust = np.zeros((3,N+1))
display_thrust = np.zeros((3,N+1))
display_allthrust = np.zeros((1,N+1))
display_allvelocity = np.zeros((1,N+1))
display_position[0:3, 0] = position_0
display_velocity[0:3, 0] = velocity_0
mass = np.zeros((1,N+1))
mass[0] = m_wet

In [ ]:

for k in range(0,N):
    v = (epsilon_k[:,k].reshape((7,1))+psi[7*k:7*(k+1),:]@eta.value)
    display_position[:,k+1] = (np.concatenate((np.identity(3), np.zeros((3,4))), axis=1)@v).reshape((3,))
    temp = np.concatenate((np.zeros((3,3)), np.identity(3)), axis=1)
    display_velocity[:,k+1] = (np.concatenate((temp, np.zeros((3,1))), axis=1)@v).reshape(3,)
    mass[0,k+1] = np.array([[0, 0, 0, 0, 0, 0, 1]])@v
    mass[0,k+1] = np.exp(mass[0,k+1])

for k in range(0,N):
    display_netthrust[:,k+1] = (np.concatenate((np.identity(3), np.zeros((3,1))), axis=1)@(upsilon[4*k:4*k+4,:]@eta.value)).reshape(3,)
    display_thrust[:,k+1] = display_netthrust[:,k+1]
    display_netthrust[:,k+1] = display_netthrust[:,k+1]*mass[0,k+1]
    #display_allthrust[0,k+1] = cp.norm(display_thrust[0:3,k+1],2)*mass[0,k+1]
    #display_allvelocity[0,k+1] = cp.norm(display_velocity[0:3,k+1],2)


In [ ]:
plt.plot(display_position[0,:],label="X")
plt.plot(display_position[1,:],label="Y")
plt.plot(display_position[2,:],label="Z")
plt.legend(loc="upper left")

In [ ]:
plt.plot(display_netthrust[0,:],label="X")
plt.plot(display_netthrust[1,:],label="Y")
plt.plot(display_netthrust[2,:],label="Z")
plt.legend(loc="upper left")

In [ ]:
m1 = plt.subplot(2,1,1)
m2 = plt.subplot(2,1,2)
m1.plot(mass[0,:],label="Z")

m2.plot(mass[0,50:52],label="Z")